In [2]:
def simulate_clock(cycles, period=1, clock = 0, prev_clock = 1):
        if clock == 1 and prev_clock == 0:
            posedgeClk = True
        elif clock == 0 and prev_clock == 1:
            posedgeClk = False
        
        return posedgeClk

In [3]:
class ProgramCounter:
    def __init__(self):
        self.pc = 0

    def updatePc(self, new_pc, rst, hold, posedgeClk):
        if rst and posedgeClk:
            self.pc = 0
        elif not hold and posedgeClk:
            self.pc = new_pc

    def __repr__(self):
        return f"PC: {self.pc}"


class InstructionMemory:
    def __init__(self, instructions, cycle):
        self.instructions = instructions
        self.cycle = cycle

    def readInstruction(self, address, rst, posedgeClk):
        if address < len(self.instructions) and not rst and posedgeClk:
            return self.instructions[address]
        else:
            return 0


class Fetch:
    def __init__(self, PCsrc, jr, jump, hold,
                 reg1Addr, jaddress, adderResult, cycle, rst, PCPlus1, posedgeClk):
        self.rst = rst
        self.cycle = cycle
        self.PCsrc = PCsrc
        self.jr = jr
        self.jump = jump
        self.hold = hold
        self.reg1Addr = reg1Addr
        self.jaddress = jaddress
        self.adderResult = adderResult
        self.posedgeClk = posedgeClk

        # Compute addresses
        self.branchAddress = adderResult if PCsrc else PCPlus1
        self.jrAddress = reg1Addr if jr else jaddress
        self.nextPC = self.jrAddress if jump else self.branchAddress

        # Initialize Program Counter
        self.pCounter = ProgramCounter()
        self.pCounter.updatePc(self.nextPC, rst, False, self.posedgeClk)
        self.PC = self.pCounter.pc
        self.PCPlus1 = self.PC + 1 if not hold else self.pCounter.pc

        # Load instructions from memory
        instructions = self.extractInstructions("imem.txt")
        self.imemory = InstructionMemory(instructions, cycle)
        self.currentInstruction = self.imemory.readInstruction(self.pCounter.pc, rst, posedgeClk = self.posedgeClk)
    
    @staticmethod
    def extractInstructions(file_path):
        binary_instructions = []
        with open(file_path, "r") as file:
            for line in file:
                parts = line.split(":")
                if len(parts) > 1:
                    binary_value = parts[1].strip().rstrip(";")
                    binary_instructions.append(int(binary_value,2))
        return binary_instructions

    def getInstruction(self):
        return bin(self.currentInstruction)

    def getPCPlus1(self):
        return self.PCPlus1


In [4]:
# Example Usage
clk = 5
fetch_unit = Fetch(
    PCsrc=False, jr=False, jump=False, hold=False,
    reg1Addr=5, jaddress=10, adderResult=15,
    cycle = clk, rst=False, PCPlus1 = 0, posedgeClk=1
)

print("Next Instruction:", fetch_unit.getInstruction())
print("PC+1:", fetch_unit.getPCPlus1())
print("PC:", fetch_unit.PC)
print("cycle:", fetch_unit.cycle)


Next Instruction: 0b100000000000011111111111111011
PC+1: 1
PC: 0
cycle: 5


In [5]:
class ControlUnit:
    _RType = 0x0 
    _addi = 0x8 
    _ori_ = 0xd
    _xori_ = 0xe
    _andi_ = 0xc
    _slti_ = 0xa
    _lw = 0x23 
    _sw = 0x2b 
    _beq = 0x4
    _j_ = 0x2
    _jal_ = 0x3
    _bne_ = 0x5
    _add_ = 0x20
    _sub_ = 0x22
    _and_ = 0x24
    _or_ = 0x25
    _slt_ = 0x2a
    _xor_ = 0x26
    _nor_ = 0x27
    _sll_ = 0x0
    _srl_ = 0x2 
    _jr_ = 0x8    

    def __init__(self, opCode, funct):
        self.opCode = opCode
        self.funct = funct
        self.RegDst = 0b0
        self.Branch = 0b0
        self.MemReadEn = 0b0
        self.MemtoReg = 0b0
        self.MemWriteEn = 0b0
        self.RegWriteEn = 0b0
        self.ALUSrc = 0b0
        self.ALUOp = 0b0
        self.bne = 0b0
        self.jump = 0b0
        self.jal = 0b0
        self.jr = 0b0

    def execute(self):
        if self.opCode == ControlUnit._RType:
            if self.funct == ControlUnit._add_:
                self.ALUOp = 0b0000
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._sub_:
                self.ALUOp = 0b0001
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._and_:
                self.ALUOp = 0b0010
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._or_:
                self.ALUOp = 0b0011
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._slt_:
                self.ALUOp = 0b0100
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._xor_:
                self.ALUOp = 0b0101
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._nor_:
                self.ALUOp = 0b0110
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._sll_:
                self.ALUOp = 0b0111
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._srl_:
                self.ALUOp = 0b1000
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._jr_:
                self.RegDst = 0b0
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.ALUOp = 0b0000
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b0
                self.ALUSrc = 0b0
                self.jump = 0b1
                self.bne = 0b0
                self.jal = 0b0
                self.jr = 0b1
        elif self.opCode == ControlUnit._addi:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0000
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._lw:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b1
            self.MemtoReg = 0b1
            self.ALUOp = 0b0000
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._sw:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0000
            self.MemWriteEn = 0b1
            self.RegWriteEn = 0b0
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._beq:
            self.RegDst = 0b0
            self.Branch = 0b1
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0001
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b0
            self.ALUSrc = 0b0
            self.jump = 0b0
            self.bne = 0b0
            self.jal = 0b0
            self.jr = 0b0
        elif self.opCode == ControlUnit._ori_:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0011
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._xori_:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0101
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1


In [6]:
cu = ControlUnit(opCode=0x0, funct=0x20)
cu.execute()
print(cu.RegWriteEn)

1


In [7]:
class RF:
    def __init__(self):
        self.registers = [0]*32
        self.readData1 = 0
        self.readData2 = 0
    def writeRegister(self, rst, writeRegister, writeData, writeEnable):
        if rst:
            self.registers = [0]*32
        elif writeEnable:
            self.registers[writeRegister] = writeData
    def readRegisters(self, readRegister1, readRegister2):
        self.readData1 = self.registers[readRegister1]
        self.readData2 = self.registers[readRegister2]
    

In [8]:
class SignExtender:
    def __init__(self):
        self.data = 0
    def signExtend(self,data):
        if data & 0x8000:
            return data - (1 << 16)  
        else: return data
        

In [9]:
class Comparator:
    def __init__(self, In1, In2, bne, rst, branch, hold):
        self.In1 = In1
        self.In2 = In2
        self.bne = bne
        self.rst = rst
        self.branch = branch
        self.hold = hold
        self.branchValid = 0
    def compare(self):
        if self.rst:
            self.branchValid = 0
        else:
            if(self.branch and not self.hold and ((self.bne and (self.In1 != self.In2)) or (not self.bne and (self.In1 == self.In2)))):
                self.branchValid = 1
            else:
                branchValid = 0
    

In [10]:
cmp = Comparator(0, 0, 0, 0, 1, 0)
cmp.compare()
print(cmp.branchValid)

1


In [11]:
class Decode:
    def __init__(self, cycle, rst, RegWriteEn_WB, jal_WB, instruction, writeData_WB, aluRes_MEM,
                 writeRegister_WB, PCPlus1, forwardA_branch, forwardB_branch, hold):
        self.cycle = cycle
        self.rst = rst
        self.RegWriteEn_WB = RegWriteEn_WB
        self.jal_WB = jal_WB
        self. instruction = instruction
        self.writeData_WB = writeData_WB
        self.aluRes_MEM = aluRes_MEM
        self.writeRegister_WB = writeRegister_WB
        self.PCPlus1 = PCPlus1
        self.forwardA_branch = forwardA_branch
        self.forwardB_branch = forwardB_branch
        self.hold = hold
        self.opCode = instruction>>26 & 0b111111
        self.rs = instruction>>21 & 0b11111
        self.rt = instruction>>16 & 0b11111
        self.rd = instruction >>11 & 0b11111
        self.imm = instruction & 0xFFFF
        self.funct = instruction & 0b111111
        self.shamt = instruction>>6 & 0b11111
        self.jaddress = instruction & 0b1111111111
    def decode(self):
        cu = ControlUnit(opCode=self.opCode, funct=self.funct)
        cu.execute()
        self.RegDst = cu.RegDst
        self.branch = cu.Branch
        self.MemReadEn = cu.MemReadEn
        self.MemtoReg = cu.MemtoReg
        self.ALUOp = cu.ALUOp
        self.MemWriteEn = cu.MemWriteEn
        self.RegWriteEn = cu.RegWriteEn
        self.ALUSrc = cu.ALUSrc
        self.bne = cu.bne
        self.jump = cu.jump
        self.jump = cu.jump
        self.jr = cu.jump
        self.DestReg = self.rt if not self.RegDst else self.rd
        self.regAddress = self.writeRegister_WB if not self.jal_WB else 0b11111
        rf = RF()
        rf.readRegisters(readRegister1=self.rs, readRegister2=self.DestReg)
        self.readData1 = rf.readData1
        self.readData2 = rf.readData2
        rf.writeRegister(rst = self.rst, writeRegister = self.regAddress,
                         writeData = self.writeData_WB, writeEnable = self.RegWriteEn_WB )
        se = SignExtender()
        self.imm = se.signExtend(self.imm)
        self.adderResult = self.PCPlus1 + self.imm
        if(self.forwardA_branch == 0):
            self.fwdA = self.readData1
        elif (self.forwardA_branch == 1):
            self.fwdA = self.aluRes_MEM
        elif (self.forwardA_branch == 2):
            self.fwdA = self.writeData_WB
        else:
            self.fwdA = 0
        if(self.forwardB_branch == 0):
            self.fwdB = self.readData2
        elif (self.forwardB_branch == 1):
            self.fwdB = self.aluRes_MEM
        elif (self.forwardB_branch == 2):
            self.fwdB = self.writeData_WB
        else:
            self.fwdB = 0
        cmp = Comparator(In1 = self.fwdA, In2 = self.fwdB, bne= self.bne, rst = self.rst, branch= self.branch, hold=0)
        cmp.compare()
        self.PCSrc = cmp.branchValid
        

    
        



   



In [12]:
instruction = 0b00010000010000110000000000001100
decode_unit = Decode(
    cycle=1,
    rst=0,
    RegWriteEn_WB=1,
    jal_WB=0,
    instruction=instruction,
    writeData_WB=123,
    aluRes_MEM=456,
    writeRegister_WB=5,
    PCPlus1=4,
    forwardA_branch=0,
    forwardB_branch=0,
    hold=0
)

decode_unit.decode()

print(f"opCode: {decode_unit.opCode}")
print(f"rs: {decode_unit.rs}")
print(f"rt: {decode_unit.rt}")
print(f"rd: {decode_unit.rd}")
print(f"funct: {decode_unit.funct}")
print(f"RegDst: {decode_unit.RegDst}")
print(f"branch: {decode_unit.branch}")
print(f"PCSrc: {decode_unit.PCSrc}")
print(f"imm: {decode_unit.imm}")
print(f"memread: {decode_unit.MemReadEn}")
print(f"pcsrc: {decode_unit.PCSrc}")

opCode: 4
rs: 2
rt: 3
rd: 0
funct: 12
RegDst: 0
branch: 1
PCSrc: 1
imm: 12
memread: 0
pcsrc: 1


In [13]:
class ALU:

    _AND  = 0b010
    _SUB  = 0b001
    _ADD = 0b000
    _OR   = 0b011
    _SLT  = 0b100 
    _XOR = 0b101
    _NOR = 0b110
    _SLL = 0b111
    _SLR = 0b1000
    
    def __init__(self, operand1, operand2, opSel, shamt):
        self.operand1 = operand1
        self.operand2 = operand2
        self.opSel = opSel
        self.shamt = shamt
        self.result = 0
    def signed(self,data):
        if data & 0x80000:
            return data - (1 << 32)  
        else: return data
        
    def execute(self):
        self.soperand1 = self.signed(self.operand1)
        self.soperand2 = self.signed(self.operand2)
        if(self.opSel == ALU._ADD):
            self.result = self.operand1 + self.operand2
        elif(self.opSel == ALU._SUB):
            self.result = self.operand1 - self.operand2
        elif(self.opSel == ALU._AND):
            self.result = self.operand1 & self.operand2
        elif(self.opSel == ALU._OR):
            self.result = self.operand1 | self.operand2
        elif(self.opSel == ALU._SLT):
            self.result = 1 if self.soperand1 < self.soperand2 else 0
        elif(self.opSel == ALU._XOR):
            self.result = self.operand1 ^ self.operand2
        elif(self.opSel == ALU._NOR):
            self.result = ~(self.operand1 | self.operand2)
        elif(self.opSel == ALU._SLL):
            self.result = self.soperand1 << self.shamt
        elif(self.opSel == ALU._SLR):
            self.result = self.soperand1 >>self.shamt 
        
        
        
            
        
            
        

    
        

In [14]:
def test_ALU():
    # Define test cases for operand1 and operand2
    test_cases = [
        {"operand1": 5, "operand2": 3, "description": "Positive operands"},
        {"operand1": -5, "operand2": -3, "description": "Negative operands"},
        {"operand1": -5, "operand2": 3, "description": "One negative, one positive"},
    ]

    # Define operations and expected outputs (description only for reference)
    operations = [
        (ALU._ADD, "Addition"),
        (ALU._SUB, "Subtraction"),
        (ALU._AND, "Bitwise AND"),
        (ALU._OR, "Bitwise OR"),
        (ALU._SLT, "Set Less Than"),
        (ALU._XOR, "Bitwise XOR"),
        (ALU._NOR, "Bitwise NOR"),
        (ALU._SLL, "Shift Left Logical"),
        (ALU._SLR, "Shift Right Logical"),
    ]

    # Test and output results
    for case in test_cases:
        print(f"--- Test: {case['description']} ---")
        for op, op_desc in operations:
            alu = ALU(operand1=case["operand1"], operand2=case["operand2"], opSel=op, shamt=2)
            alu.execute()
            print(f"{op_desc} ({alu.operand1}, {alu.operand2}): Result = {alu.result}")
        print()

# Run the tests
test_ALU()


--- Test: Positive operands ---
Addition (5, 3): Result = 8
Subtraction (5, 3): Result = 2
Bitwise AND (5, 3): Result = 1
Bitwise OR (5, 3): Result = 7
Set Less Than (5, 3): Result = 0
Bitwise XOR (5, 3): Result = 6
Bitwise NOR (5, 3): Result = -8
Shift Left Logical (5, 3): Result = 20
Shift Right Logical (5, 3): Result = 1

--- Test: Negative operands ---
Addition (-5, -3): Result = -8
Subtraction (-5, -3): Result = -2
Bitwise AND (-5, -3): Result = -7
Bitwise OR (-5, -3): Result = -1
Set Less Than (-5, -3): Result = 1
Bitwise XOR (-5, -3): Result = 6
Bitwise NOR (-5, -3): Result = 0
Shift Left Logical (-5, -3): Result = -17179869204
Shift Right Logical (-5, -3): Result = -1073741826

--- Test: One negative, one positive ---
Addition (-5, 3): Result = -2
Subtraction (-5, 3): Result = -8
Bitwise AND (-5, 3): Result = 3
Bitwise OR (-5, 3): Result = -5
Set Less Than (-5, 3): Result = 1
Bitwise XOR (-5, 3): Result = -8
Bitwise NOR (-5, 3): Result = 4
Shift Left Logical (-5, 3): Result = -

In [22]:
class Execute:
    def __init__(self, ALUSrc, forwardA, forwardB, ALUOp, shamt_EX, readData1, readData2, aluRes_MEM, aluRes_WB, extImm):
        self.ALUSrc = ALUSrc
        self.forwardA = forwardA
        self.forwardB = forwardB
        self.ALUOp = ALUOp
        self.shamt_EX = shamt_EX
        self.readData1 = readData1
        self.readData2 = readData2
        self.aluRes_MEM = aluRes_MEM
        self.aluRes_WB = aluRes_WB
        self.extImm = extImm
        self.aluRes_EX = 0
        self.forwardBRes = 0
    def execute(self):
        if(self.forwardA == 0):
            self.forwardARes = self.readData1
        elif(self.forwardA == 1):
            self.forwardARes = self.aluRes_WB
        elif(self.forwardA == 2):
            self.forwardARes = self.aluRes_MEM
        else:
            self.forwardARes = 0
            
        if(self.forwardB== 0):
            self.forwardBRes = self.readData2
        elif(self.forwardB == 1):
            self.forwardBRes = self.aluRes_WB
        elif(self.forwardB == 2):
            self.forwardBRes = self.aluRes_MEM
        else:
            self.forwardBRes = 0
        self.ALUin2 = self.extImm if self.ALUSrc else self.forwardBRes
        alu = ALU(operand1=self.forwardARes, operand2=self.ALUin2, opSel=self.ALUOp, shamt = self.shamt_EX)
        alu.execute()
        self.aluRes_EX = alu.result
        
        

In [26]:

if __name__ == "__main__":
    test_cases = [
        # Test Case 1: Forwarding from readData1, ALUSrc = 0 (use forwardBRes)
        {
            "ALUSrc": 0, "forwardA": 0, "forwardB": 0, "ALUOp": 0, "shamt_EX": 0,
            "readData1": 10, "readData2": 20, "aluRes_MEM": 0, "aluRes_WB": 0, "extImm": 5,
            "expected_result": 30  # ADD: 10 + 20
        },
        # Test Case 2: Forwarding from aluRes_WB, ALUSrc = 1 (use extImm)
        {
            "ALUSrc": 1, "forwardA": 1, "forwardB": 0, "ALUOp": 1, "shamt_EX": 0,
            "readData1": 10, "readData2": 20, "aluRes_MEM": 0, "aluRes_WB": 15, "extImm": 7,
            "expected_result": 8  # SUB: 15 - 7
        },
        # Test Case 3: Forwarding from aluRes_MEM, ALUSrc = 0 (use forwardBRes)
        {
            "ALUSrc": 0, "forwardA": 2, "forwardB": 2, "ALUOp": 2, "shamt_EX": 0,
            "readData1": 10, "readData2": 20, "aluRes_MEM": 12, "aluRes_WB": 0, "extImm": 5,
            "expected_result": 12  # AND: 12 & 12
        },
        # Test Case 4: ALU operation with immediate value and SLL operation
        {
            "ALUSrc": 1, "forwardA": 0, "forwardB": 0, "ALUOp": 7, "shamt_EX": 2,
            "readData1": 3, "readData2": 0, "aluRes_MEM": 0, "aluRes_WB": 0, "extImm": 3,
            "expected_result": 12  # SLL: 3 << 2
        },
        # Test Case 5: No forwarding and SRL operation
        {
            "ALUSrc": 1, "forwardA": 0, "forwardB": 0, "ALUOp": 8, "shamt_EX": 1,
            "readData1": 4, "readData2": 0, "aluRes_MEM": 0, "aluRes_WB": 0, "extImm": 4,
            "expected_result": 2  # SRL: 4 >> 1
        },
    ]

    for i, test in enumerate(test_cases, 1):
        execute_unit = Execute(
            ALUSrc=test["ALUSrc"],
            forwardA=test["forwardA"],
            forwardB=test["forwardB"],
            ALUOp=test["ALUOp"],
            shamt_EX=test["shamt_EX"],
            readData1=test["readData1"],
            readData2=test["readData2"],
            aluRes_MEM=test["aluRes_MEM"],
            aluRes_WB=test["aluRes_WB"],
            extImm=test["extImm"]
        )

        execute_unit.execute()
        

        assert execute_unit.aluRes_EX == test["expected_result"], (
            f"Test case {i} failed: expected {test['expected_result']}, got {execute_unit.aluRes_EX}"
        )

        print(f"Test case {i} passed!")


Test case 1 passed!
Test case 2 passed!
Test case 3 passed!
Test case 4 passed!
Test case 5 passed!


In [28]:
class MEMORY:
    def __init__(self, aluRes_MEM, WriteDmem, MemReadEn_MEM, MemWriteEn_MEM):
        self.aluRes_MEM = aluRes_MEM
        self.WriteDmem = WriteDmem
        self.MemReadEn_MEM = MemReadEn_MEM
        self.MemWriteEn_MEM = MemWriteEn_MEM
        self.memoryReadData = 0
        self.memRows = [0]*256
    def initializeFromFile(self, file_path):
        with open(file_path, "r") as file:
            for index, line in enumerate (file):
                if(index < len(self.memRows)):
                    binary_value = line.strip()
                    self.memRows[index] = int(binary_value, 2)
                else: 
                    break 
        
        

In [34]:
mem = MEMORY(aluRes_MEM=1, WriteDmem=0, MemReadEn_MEM=0, MemWriteEn_MEM=0)
mem.initializeFromFile("dmem.txt")
for i in range(10):
    print(bin(mem.memRows[i]))  #FYI python cannot read stuff in binary, 
                                # but it does make them integer, 
                                # hence, don't think too much about if 
                                # something is 32 bits or not

0b1
0b10
0b11
0b100
0b101
0b110
0b111
0b0
0b0
0b0
